# Prática - Modulo 1 - IGTI - Data Engineering

### Autor: Thomaz Antonio Rossito Neto                                        
### LinkedIn: [clique aqui](https://www.linkedin.com/in/thomaz-antonio-rossito-neto/)

<img src="https://notiserrasc.com.br/wp-content/uploads/2019/10/enem2019-680x350.jpg">

## - Parte 1: Extract and Transform

> #### Enunciado

Você é uma pessoa da área de Engenheira de Dados em uma empresa de consultoria
educacional. Os consultores da empresa vão auxiliar um grupo educacional de ensino
médio a desenvolver seu planejamento pedagógico e suas metodologias de ensino.
Nesse sentido, eles fizeram a você uma requisição de preparar dados do ENEM para
análise.

> #### Objetivo

Você deve, portanto, extrair os dados do ENEM, selecionar apenas os alunos de Minas
Gerais e apresentar algumas análises que vão ajudar o trabalho dos consultores.

> #### Atividades

Você deverá desempenhar as seguintes atividades:

1. Extrair, de maneira programática, os dados do ENEM 2019.
2. Filtrar apenas os alunos do estado de Minas Gerais.
3. Salvar um arquivo CSV com os dados já limpos em seu computador.
4. Escrever os dados numa tabela relacional em uma base de dados de sua escolha
    (MySQL, PostgreSQL, SQL Server etc.).

## - Inínio do Trabalho

### 1. Carregando as bibliotecas

In [1]:
# Imports de avisos
import sys
import os
import gc
import zipfile
import requests
from io import BytesIO

# Imports para manipulação de dados
import numpy as np
import pandas as pd
import datatable
from datatable import f

### 2. Criando funções que serão utilizadas neste notebook

In [2]:
# Funcao pare retornar a utilizacao de memoria
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 

# Funcao para retornar o tamanho do dataset
def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

# Funcao para redução do uso de memoria
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

### 3. Criar um diretório para armazenar o conteúdo do ENEM

In [3]:
os.makedirs('./enem2019', exist_ok=True)

### 4. Importando o dataset do site do INEP e extraindo para o diretório enem2019

In [ ]:
# Atribuindo o link a um objeto 'url'
url = "http://download.inep.gov.br/microdados/microdados_enem_2019.zip"

# Faz o download do conteúdo 
filebytes = BytesIO(
    requests.get(url).content
)

# Extrair o conteúdo do 'zipfile'
myzip = zipfile.ZipFile(filebytes)
myzip.extractall("./enem2019")

### 5. Carregando o dataset do Enem 2019 e visualizando informações do Dataset

In [5]:
# Carregando o dataset completo usando o pacote datatable
df = datatable.fread('enem2019/DADOS/MICRODADOS_ENEM_2019.csv', sep=';', encoding='latin-1')

# Filtrando apenas alunos do estado de Minas Gerais
df = df[f.SG_UF_RESIDENCIA == 'MG', :]

# Convertendo para dataframe Pandas
df = df.to_pandas()

print()
# Verificando o uso da memoria
print("{:}: {:>8}".format('Dataset Original',sizeof_fmt(df.memory_usage(index=True).sum())))

print()
# Reduzindo tamanho do dataset em memoria
df = reduce_mem_usage(df)

print()
# Verificando o uso da memoria depois da reducao
print("{:}: {:>9}".format('Dataset Reduzido',sizeof_fmt(df.memory_usage(index=True).sum())))

print()
# Visualizando o shape do dataset
print("Shape: ", df.shape)


Dataset Original: 333.5MiB

Mem. usage decreased to 240.01 Mb (28.0% reduction)

Dataset Reduzido:  240.0MiB

Shape:  (538896, 136)


In [6]:
# Visualizando os primeiros registros
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,...,B,A,A,D,A,A,D,B,B,B
1,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,...,A,A,A,B,A,A,C,A,A,A
2,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,...,A,A,A,C,B,A,C,A,A,B
3,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,...,B,A,B,B,A,A,D,A,D,B
4,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,...,A,A,A,B,B,A,C,A,B,A


In [7]:
# Limpando a memoria
print("gc: ", gc.collect())

gc:  214


### 6. Comprimir ainda mais o arquivo e salvar em formato zip 

In [8]:
compression_opts = dict(method='zip', archive_name='enem_2019_MG.csv')  
df.to_csv('enem2019/DADOS/enem_2019_MG.zip', index=False, compression=compression_opts)

> ## Agora temos um arquivo "`.zip`" que deve estar em torno de 71MB

### 7. Confirmando a leitura do dataset

In [9]:
df = pd.read_csv('enem2019/DADOS/enem_2019_MG.zip')
df.head()

,NU_INSCRICAO,NU_ANO,CO_MUNICIPIO_RESIDENCIA,NO_MUNICIPIO_RESIDENCIA,CO_UF_RESIDENCIA,SG_UF_RESIDENCIA,NU_IDADE,TP_SEXO,TP_ESTADO_CIVIL,TP_COR_RACA,...,Q016,Q017,Q018,Q019,Q020,Q021,Q022,Q023,Q024,Q025
0,190001005671,2019,3170206,Uberlândia,31,MG,21,M,1,1,...,B,A,A,D,A,A,D,B,B,B
1,190001009861,2019,3111705,Canaã,31,MG,20,M,1,1,...,A,A,A,B,A,A,C,A,A,A
2,190001021622,2019,3104007,Araxá,31,MG,24,M,1,2,...,A,A,A,C,B,A,C,A,A,B
3,190001026636,2019,3106200,Belo Horizonte,31,MG,33,M,1,1,...,B,A,B,B,A,A,D,A,D,B
4,190001026742,2019,3162401,São João da Ponte,31,MG,23,M,1,3,...,A,A,A,B,B,A,C,A,B,A


# FIM...
### WebSite Pessoal: [thomazrossito](https://www.thomazrossito.com.br)